This code is derived from these repositories:

https://github.com/xingjiepan/MERFISH_analysis/tree/main/cell_cell_contact

https://github.com/ZhuangLab/whole_mouse_brain_MERFISH_atlas_scripts_2023/blob/main/scripts/cell_cell_contacts/get_significant_contacts_30um.ipynb

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import anndata
import scanpy as sc
sc.settings.n_jobs = 56
sc.settings.set_figure_params(dpi=180, dpi_save=300, frameon=False, figsize=(4, 4), fontsize=8, facecolor='white')

import sys
from permutation import generate_cell_type_contact_count_matrices
import scipy.cluster
import scipy.stats
import statsmodels.stats.multitest

import random
random.seed(42)

In [2]:
adata = sc.read_h5ad('../../MERFISH/Baysor/ABC_cleaned.h5ad')

#wanting to make sure we only show the neurons with a high confidence
adata_HQ = adata[adata.obs.subclass_label_confidence > 0.8]

adata_HQ.X = adata_HQ.layers['counts'].toarray().copy()
sc.pp.normalize_total(adata_HQ)
sc.pp.log1p(adata_HQ)

# just placing the region-age pairs in .csvs to look at later
for ages in adata_HQ.obs.Age.unique():
    for regions in adata_HQ.obs.Brain_Region.unique():
        adata_HQ[(adata_HQ.obs.Age == ages) & (adata_HQ.obs.Brain_Region == regions)].obs.to_csv(f'contacts/{regions + ages}.csv')

/home/doug.henze/.conda/envs/Vizgen_2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/doug.henze/.conda/envs/Vizgen_2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


In [3]:
output_path = 'outputs_30um'
os.makedirs(output_path, exist_ok=True)

In [4]:
# This code is heavily adapted from the cited repos
major_brain_regions = ["Cerebellum","Hippocampus"]
ages = ['3','24']

for region in major_brain_regions:
    for age in ages:
        print(region)
    
        # Read the data
        df_ct_labels = pd.read_csv(os.path.join('contacts', f'{region+age}.csv'), index_col=0)
        slice_ids = np.unique(df_ct_labels['batchID'])
    
        cell_type_col = 'subclass_label_transfer'
        cell_types = np.unique(df_ct_labels[cell_type_col])
        N_cell_types = len(cell_types)
        N_permutations = 1000
    
    # Count and save the contacts without permutation
        merged_contact_counts = np.zeros((N_cell_types, N_cell_types), dtype=int)

        for slice_id in slice_ids:
            print(f'Count for {slice_id}')
            df_slice = df_ct_labels[df_ct_labels['batchID'] == slice_id]
            cell_type_contact_counts = generate_cell_type_contact_count_matrices(df_slice, cell_type_col, 
                                        ['aligned_x', 'aligned_y'], cell_types, 
                                        permutation_method='no_permutation', contact_radius=30)
    
            merged_contact_counts = merged_contact_counts + cell_type_contact_counts
    
        output_file = os.path.join(output_path, f'{region+age}_no_permutation.npy')
        np.save(output_file, merged_contact_counts)
    
        from multiprocessing import Pool

        def permute_and_count_contacts_for_slices(df_slice_list):
            merged_contact_counts = np.zeros((N_permutations, N_cell_types, N_cell_types), dtype=int)
            for df_slice in df_slice_list:
            
                for i in range(N_permutations):
                    df_slice_rand = df_slice.copy()
                
                    r_permute = 100
                    r = r_permute * np.sqrt(np.random.uniform(size=df_slice_rand.shape[0]))
                    theta = np.random.uniform(size=df_slice_rand.shape[0]) * 2 * np.pi
                
                    df_slice_rand['aligned_x'] += r * np.sin(theta)
                    df_slice_rand['aligned_y'] += r * np.cos(theta)
            
                    cell_type_contact_counts = generate_cell_type_contact_count_matrices(df_slice_rand, cell_type_col, 
                                        ['aligned_x', 'aligned_y'], cell_types, 
                                        permutation_method='no_permutation', contact_radius=30)
            
            
                    merged_contact_counts[i] = merged_contact_counts[i] + cell_type_contact_counts
        
            return merged_contact_counts
    
    # Get the dataframe for each slice
        all_df_slice_list = [df_ct_labels[df_ct_labels['batchID'] == slice_id] for slice_id in slice_ids]

    # Split the slices into groups
        N_groups = 48
        group_size = int(np.ceil(len(slice_ids) / N_groups))
        grouped_slice_dfs = []

        for i in range(N_groups):
            slice_id_start = i * group_size
            slice_id_stop = (i + 1) * group_size
            grouped_slice_dfs.append(all_df_slice_list[slice_id_start:slice_id_stop])
    
    # Permute and count the contacts in parallel
        N_permutations = 1000
        with Pool(N_groups) as p:
            contact_analysis_results = p.map(permute_and_count_contacts_for_slices, grouped_slice_dfs)
    
        merged_contact_counts = np.sum(contact_analysis_results, axis=0)
    
        means = np.mean(merged_contact_counts, axis=0)
        stds = np.std(merged_contact_counts, axis=0)
    
        np.save(os.path.join(output_path, f'{region+age}_local_permutation_count_tensor.npy'),
            merged_contact_counts)
    
        output_file_mean = os.path.join(output_path, f'{region+age}_local_permutation_mean.npy')
        np.save(output_file_mean, means)
        output_file_std = os.path.join(output_path, f'{region+age}_local_permutation_std.npy')
        np.save(output_file_std, stds)

Cerebellum
Count for 3-mo-female-1-rev2
Count for 3-mo-female-2
Count for 3-mo-female-3
Count for 3-mo-male-1
Count for 3-mo-male-2
Count for 3-mo-male-3-rev2
Cerebellum
Count for 24-mo-female-1
Count for 24-mo-female-3
Count for 24-mo-female-5
Count for 24-mo-male-2
Count for 24-mo-male-4-rev2
Hippocampus
Count for 3-mo-female-1-rev2
Count for 3-mo-female-2
Count for 3-mo-female-3
Count for 3-mo-male-2
Count for 3-mo-male-3-rev2
Hippocampus
Count for 24-mo-female-1
Count for 24-mo-female-3
Count for 24-mo-female-5
Count for 24-mo-male-1
Count for 24-mo-male-2
Count for 24-mo-male-4-rev2


In [5]:
def count_zero_pairs(contact_mtx):
    n_0 = 0
    for i in range(contact_mtx.shape[0]):
        for j in range(i, contact_mtx.shape[0]):
            if contact_mtx[i, j] == 0:
                n_0 += 1
    return n_0

def adjust_p_value_matrix_by_BH(p_val_mtx):
    '''Adjust the p-values in a matrix by the Benjamini/Hochberg method.
    The matrix should be symmetric.
    '''
    p_val_sequential = []
    N = p_val_mtx.shape[0]
    
    for i in range(N):
        for j in range(i, N):
            p_val_sequential.append(p_val_mtx[i, j])

    p_val_sequential_bh = statsmodels.stats.multitest.multipletests(p_val_sequential, method='fdr_bh')[1]
    
    adjusted_p_val_mtx = np.zeros((N, N))
    
    counter = 0
    for i in range(N):
        for j in range(i, N):
            adjusted_p_val_mtx[i, j] = p_val_sequential_bh[counter]
            adjusted_p_val_mtx[j, i] = p_val_sequential_bh[counter]
            counter += 1
            
    return adjusted_p_val_mtx

def get_data_frame_from_metrices(cell_types, mtx_dict):
    N = len(cell_types)
    
    serials_dict = {'cell_type1':[], 'cell_type2':[]}
    for k in mtx_dict.keys():
        serials_dict[k] = []
        
    for i in range(N):
        for j in range(i, N):
            serials_dict['cell_type1'].append(cell_types[i])
            serials_dict['cell_type2'].append(cell_types[j])
            for k in mtx_dict.keys():
                serials_dict[k].append(mtx_dict[k][i, j])
                
    return pd.DataFrame(serials_dict)
    

def sort_cell_type_contact_p_values(p_val_mtx, cell_types):
    '''Return a list of (cell_type1, cell_type2, p_value) sorted by p_values.'''
    p_val_list = []
    N = p_val_mtx.shape[0]
    for i in range(N):
        for j in range(i, N):
            p_val_list.append((cell_types[i], cell_types[j], p_val_mtx[i, j]))
    return sorted(p_val_list, key=lambda x:x[2])
def get_optimal_order_of_mtx(X):
    Z = scipy.cluster.hierarchy.ward(X)
    return scipy.cluster.hierarchy.leaves_list(
        scipy.cluster.hierarchy.optimal_leaf_ordering(Z, X))

def get_ordered_tick_labels(tick_labels):
    tick_labels_with_class = [s.split(' ')[-1] + ' ' + s for s in tick_labels]
    return np.argsort(tick_labels_with_class)

def filter_pval_mtx(pval_mtx, tick_labels, allowed_pairs):
    pval_mtx_filtered = pval_mtx.copy()
    
    for i in range(pval_mtx.shape[0]):
        ct1 = tick_labels[i]
        for j in range(pval_mtx.shape[1]):
            ct2 = tick_labels[j]
            
            if ((ct1, ct2) in allowed_pairs) or ((ct2, ct1) in allowed_pairs):
                continue
            else:
                pval_mtx_filtered[i, j] = 1
            
    return pval_mtx_filtered

In [6]:
# Now that we created our .npy files we can utilize them to count our observed vs theoretical contacts
# This code is also heavily inspired by the cited repos
permutation_path = 'outputs_30um'

major_brain_regions = ["Cerebellum","Hippocampus"]

result_dfs = []

for region in major_brain_regions:
    for age in ages:
    
    # Load the cell type labels
        df_ct_labels = pd.read_csv(os.path.join('contacts', f'{region+age}.csv'), index_col=0)

        subclass_types = np.unique(df_ct_labels['subclass_label_transfer'])
    
        cell_contact_counts = np.load(os.path.join(permutation_path, f'{region+age}_no_permutation.npy'))

        local_null_means = np.load(os.path.join(permutation_path, f'{region+age}_local_permutation_mean.npy'))
        local_null_stds = np.load(os.path.join(permutation_path, f'{region+age}_local_permutation_std.npy'))

    # Require all stds to be larger or equal to the minimal observable std value
        local_null_stds = np.maximum(local_null_stds, np.sqrt(1 / 1000))
    
        local_z_scores = (cell_contact_counts - local_null_means) / local_null_stds
        local_p_values = scipy.stats.norm.sf(local_z_scores)
        adjusted_local_p_values = adjust_p_value_matrix_by_BH(local_p_values)
    
        fold_changes = cell_contact_counts / (local_null_means + 1e-4)
        filled_region = np.full(adjusted_local_p_values.shape, region)
        filled_age = np.full(adjusted_local_p_values.shape, age)
    
    #make_dotplot(local_p_values, fold_changes, subclass_types, title=region)
    #make_dotplot(adjusted_local_p_values, fold_changes, subclass_types, title=region)
    #make_dotplot(adjusted_local_p_values, fold_changes, subclass_types, title=region + ' L-R filtered', 
    #             allowed_pairs=allowed_pairs)
        
    # Gather all results into a data frame
        contact_result_df = get_data_frame_from_metrices(subclass_types, 
                                             {'pval-adjusted': adjusted_local_p_values,
                                              'pval': local_p_values,
                                              'z_score': local_z_scores,
                                              'contact_count': cell_contact_counts,
                                              'permutation_mean': local_null_means,
                                              'permutation_std': local_null_stds,
                                              'region': filled_region,
                                              'age': filled_age
                                            }).sort_values('z_score', ascending=False)

    # Filter out pairs that don't contact
    # notably we are not evaluating p-values yet
        contact_result_df = contact_result_df[contact_result_df['contact_count'] > 0]
        contact_result_df.to_csv(os.path.join(permutation_path, f'{region+age}_close_contacts.csv'))
    
        result_dfs.append(contact_result_df)

In [7]:
# Now I want to compare the difference between 24 and 3 versus what we would expect it to be randomly
# We will then use this to filter p-values

combined_results = pd.concat(result_dfs)
df_age3 = combined_results[combined_results['age'] == '3']
df_age24 = combined_results[combined_results['age'] == '24']

# Merge the dataframes on cell_type1, cell_type2, and region
merged_df = pd.merge(df_age3, df_age24, on=['cell_type1', 'cell_type2', 'region'], suffixes=('_3', '_24'))

# Calculate the z-statistic for the differences in contact_count
merged_df['z_statistic'] = (merged_df['contact_count_24'] - merged_df['contact_count_3']) / \
                           (merged_df['permutation_std_3']**2 + merged_df['permutation_std_24']**2)**0.5
merged_df['p_value'] = scipy.stats.norm.sf(abs(merged_df['z_statistic'])) * 2

merged_df['p_value_adj'] = statsmodels.stats.multitest.multipletests(merged_df['p_value'].values, method='fdr_bh')[1]

merged_df_trim = merged_df[merged_df.p_value_adj < 0.05]

In [8]:
# We want to save this new dataframe on a per region basis
# new output permutation path
permutation_path = 'outputs_30um_age'
os.makedirs(permutation_path, exist_ok=True)
for region in major_brain_regions:
    contact_result_df = merged_df_trim[merged_df_trim.region == region]
    contact_result_df.to_csv(os.path.join(permutation_path, f'{region}_close_contacts.csv'))

# Now we want to count contacts for the Cerebellum and Hippocampus

In [9]:
Cerebellum = pd.read_csv('outputs_30um_age/Cerebellum_close_contacts.csv', index_col=0)
Hippocampus = pd.read_csv('outputs_30um_age/Hippocampus_close_contacts.csv', index_col=0)

Cerebellum['Region'] = 'Cerebellum'
Hippocampus['Region'] = 'Hippocampus'

In [10]:
from scipy.stats import norm
def calculate_ratio_distribution(df, dist1_mean_col = 'permutation_mean_24', dist1_std_col = 'permutation_std_24', dist2_mean_col = 'permutation_mean_3',
                                 dist2_std_col = 'permutation_std_3',real_mean_1='contact_count_24',real_mean_2='contact_count_3'):
    """
    Calculate the mean and standard deviation of the ratio of two distributions.

    Parameters:
    df (pd.DataFrame): Input DataFrame containing distribution parameters.
    dist1_mean_col (str): Column name for the mean of distribution 1.
    dist1_std_col (str): Column name for the standard deviation of distribution 1.
    dist2_mean_col (str): Column name for the mean of distribution 2.
    dist2_std_col (str): Column name for the standard deviation of distribution 2.

    Returns:
    pd.DataFrame: DataFrame with additional columns `ratio_mean` and `ratio_std`.
    """
    # Extract values
    dist1_mean = df[dist1_mean_col]
    dist1_std = df[dist1_std_col]
    dist2_mean = df[dist2_mean_col]
    dist2_std = df[dist2_std_col]
    
    real1_mean = df[real_mean_1]
    real2_mean = df[real_mean_2]

    # Compute ratio mean    
    ratio_mean = dist1_mean / dist2_mean
    ratio_real = real1_mean / real2_mean

    # Compute ratio standard deviation using the delta method
    ratio_std = np.sqrt(
        (dist1_std / dist2_mean) ** 2 + 
        (dist1_mean * dist2_std / dist2_mean ** 2) ** 2
    )

    # Calculate z-scores
    z_score = (ratio_real - ratio_mean) / ratio_std

    # Calculate two-tailed p-values
    p_value = 2 * (1 - norm.cdf(np.abs(z_score)))

    # Add results to the DataFrame
    df = df.copy()  # Avoid modifying the original DataFrame
    df['ratio_real'] = ratio_real
    df['ratio_mean'] = ratio_mean
    df['ratio_std'] = ratio_std
    df['z_score'] = z_score
    df['p_value'] = p_value

    return df

In [11]:
# values were exported and plotted in Prism as ratio_mean and ratio_std
CB.to_csv('Cerebellum_proximity_scores.csv')
HP.to_csv('Hippocampus_proximity_scores.csv')

NameError: name 'CB' is not defined